In [1]:
import polars as pl
from pathlib import Path
import re
import json

In [3]:
# data_path = Path(r"C:\Users\user\Project\SideProjects\RAG\data\patent_samples.csv")
data_path = Path(r"C:\Users\User\Projects\RAG\data\patent_samples.csv")
df = pl.read_csv(data_path)
df


id,app_doc_number,invention_title,abstract,description,claims
i64,i64,str,str,str,str
415065,13538195,"""Programmable low power multi-m…",""" Disclosed is a method and ap…",""" BACKGROUND 1. Field of th…",""" 1. A multi-modulus divider…"
550748,13248165,"""Hearing aid device for frequen…",""" With a hearing aid device su…",""" CROSS-REFERENCE TO RELATED …",""" 1. A method for frequency …"
110416,12569987,"""Battery cooling apparatus for …",""" A battery pack apparatus has…",""" BACKGROUND The present di…",""" 1. A battery pack apparatu…"
619075,13656105,"""Methods of inhibiting cyanobac…",""" This invention offers an eff…",""" BACKGROUND OF THE INVENTION…",""" 1. A method of inhibiting …"
597454,14104237,"""Method for welding gold-silico…",""" Relating to electronic compo…",""" CROSS-REFERENCE TO RELATED …",""" 1. A method for welding a …"
…,…,…,…,…,…
468352,13335140,"""Continuous recovery system for…",""" A continuous recovery system…",""" FEDERALLY SPONSORED RESEARC…",""" 1. A continuous recovery s…"
374985,13762177,"""Cement compositions with a hig…",""" A method of cementing in a s…",""" CROSS-REFERENCE TO RELATED …",""" 1. A cement composition co…"
316450,13135745,"""Use of titanium-based material…",""" Compositions containing meta…",""" RELATED APPLICATIONS This…",""" 1. A process for treating …"


In [4]:
def preprocess_str(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    cleaned_text = ''.join(c for c in cleaned_text if c.isprintable())
    cleaned_text = cleaned_text.strip()
    return cleaned_text

In [5]:
new_list = []
for i, row in enumerate(df.iter_rows(named=True)):
    new_dict = {'id': row['id'], 'doc_num': row['app_doc_number'], 'title': row['invention_title'],
                'abstract': preprocess_str(row['abstract']), 'description': preprocess_str(row['description']),
                'claims': preprocess_str(row['claims'])}
    new_list.append(new_dict)

In [6]:
len(new_list)

5000

In [7]:
new_df_cols = ['id', 'doc_num', 'title', 'abstract', 'description', 'claims']
new_df = pl.DataFrame(new_list, schema=new_df_cols)
new_df

id,doc_num,title,abstract,description,claims
i64,i64,str,str,str,str
415065,13538195,"""Programmable low power multi-m…","""Disclosed is a method and appa…","""BACKGROUND 1. Field of the Dis…","""1. A multi-modulus divider (MM…"
550748,13248165,"""Hearing aid device for frequen…","""With a hearing aid device suit…","""CROSS-REFERENCE TO RELATED APP…","""1. A method for frequency comp…"
110416,12569987,"""Battery cooling apparatus for …","""A battery pack apparatus has a…","""BACKGROUND The present disclos…","""1. A battery pack apparatus co…"
619075,13656105,"""Methods of inhibiting cyanobac…","""This invention offers an effec…","""BACKGROUND OF THE INVENTION 1.…","""1. A method of inhibiting the …"
597454,14104237,"""Method for welding gold-silico…","""Relating to electronic compone…","""CROSS-REFERENCE TO RELATED APP…","""1. A method for welding a gold…"
…,…,…,…,…,…
468352,13335140,"""Continuous recovery system for…","""A continuous recovery system f…","""FEDERALLY SPONSORED RESEARCH O…","""1. A continuous recovery syste…"
374985,13762177,"""Cement compositions with a hig…","""A method of cementing in a sub…","""CROSS-REFERENCE TO RELATED APP…","""1. A cement composition compri…"
316450,13135745,"""Use of titanium-based material…","""Compositions containing metal …","""RELATED APPLICATIONS This appl…","""1. A process for treating oral…"


In [8]:
with open("patent_samples.json", "w", encoding='utf-8') as f:
    json.dump(new_list, f, ensure_ascii=False, indent=4)